
# TOPTW

This notebook is meant as a starter for the assignment. It **reads the classic TOPTW test instance format** (as used by the Righini & Salani set) and provides:
- Clear documentation of the file structure.
- A robust parser for the variable-length lines.
- Helpers to build distance/time matrices with the **rounding-down rule** used in the literature.
- A recap of the **mathematical model** (hard vs soft time windows) you will optimize.
- Pointer to the **benchmark instances**.

> Benchmark hub (TOP / TOPTW): **KU Leuven CIB**  
> https://www.mech.kuleuven.be/en/cib/op#autotoc-item-autotoc-6



## Mathematical model (TOPTW) — hard & soft time windows

Let the depot be node $0$, customers $V=\{1,\dots,n\}$, and vehicles $k=1,\dots,K$.
Let $d_{ij}$ be distances and $t_{ij}$ travel times (assume $t_{ij}=d_{ij}$), $q_i$ demand (0 if not used), $s_i$ service time,
$[a_i,b_i]$ time window, and $p_i$ profit. Capacity $Q$, optional fixed cost $f$ per vehicle, lateness penalty $\beta\ge 0$.
Big-$M$ constants $M_t,M_q$ deactivate time/load propagation on unused arcs.

**Decision variables**
- $x_{ijk}\in\{0,1\}$: arc $(i\to j)$ is used by vehicle $k$ (with $i\neq j$).
- $y_i\in\{0,1\}$: customer $i$ is served.
- $u_k\in\{0,1\}$: vehicle $k$ is used.
- $t_i\ge 0$: service start time at node $(i)$.
- $w_i\ge 0$: load after serving $(i)$.
- (**soft-TW only**) $L_i\ge 0$: lateness at $i$.

**Objective (hard time windows)**
$$
\max \sum_{i\in V} p_i y_i - \sum_{k=1}^K \sum_{\substack{i,j\in\{0\}\cup V\\ i\ne j}} d_{ij} x_{ijk} - f\sum_{k=1}^K u_k.
$$

**Objective (soft time windows)**
$$
\max \sum_{i\in V} p_i y_i - \sum_{k=1}^K \sum_{\substack{i,j\in\{0\}\cup V\\ i\ne j}} d_{ij} x_{ijk} - f\sum_{k=1}^K u_k - \beta \sum_{i\in V} L_i.
$$

**Constraints**
- Vehicle usage and depot degree (per vehicle):
$$
\sum_{j\in V} x_{0jk}=\sum_{i\in V} x_{i0k}=u_k \quad \forall k.
$$
- Visit equals served (flow over all vehicles):
$$
\sum_{j\in\{0\}\cup V}\sum_{k=1}^K x_{ijk}=\sum_{j\in\{0\}\cup V}\sum_{k=1}^K x_{jik}=y_i \quad \forall i\in V.
$$
- Capacity propagation (optional if no capacity is modeled):
$$
w_j \ge w_i + q_j - M_q\!\left(1-\sum_{k=1}^K x_{ijk}\right),\qquad 0\le w_i \le Q\,y_i.
$$
- Time propagation:
$$
t_j \ge t_i + s_i + t_{ij} - M_t\!\left(1-\sum_{k=1}^K x_{ijk}\right).
$$
- Time windows (choose one policy):
  - **Hard TW**: $a_i \le t_i \le b_i + M_t(1-y_i)$.
  - **Soft TW**: $a_i \le t_i \le b_i + L_i + M_t(1-y_i), \; L_i\ge 0$.
- Optional route duration per vehicle $k$: $t^{\text{return},k}_0 \le t^{\text{start},k}_0 + T_{\max}$.

*This compact VRPTW-style formulation is the one used in the assignment.*


In [1]:
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class TOPTWNode:
    i: int
    x: float
    y: float
    service: float    # service duration
    profit: float     # profit / score
    tw_open: float    # earliest service time
    tw_close: float   # latest service time
    f: int            # extra field from original format
    a: int            # length of aux_list in original file
    aux_list: List[int]
    demand: int = 0   # NEW: demand of this node (0 for depot)

@dataclass
class TOPTWInstance:
    path: str
    k: int            # number of vehicles / routes allowed
    v: int            # carried over from source format
    N: int            # number of customers (excl. depot)
    t: int            # carried over from source format
    D: Optional[float]
    Q: Optional[float]   # vehicle capacity we'll assign
    Tmax: float          # usually depot.tw_close in source data
    nodes: List[TOPTWNode]


In [2]:
import math
import pathlib
import re
from typing import Tuple

def _floor_to_decimals(x: float, decimals: int) -> float:
    if decimals is None:
        return float(x)
    f = 10 ** decimals
    return math.floor(x * f) / f

def _auto_rounding_decimals_from_name(path: str) -> int:
    # c101, r201, rc2xx -> Solomon (1 decimal); else Cordeau (2 decimals)
    name = pathlib.Path(path).name.lower()
    if re.search(r'(?:^|_)((c|r|rc)\d+)', name):
        return 1
    return 2

def build_distance_time_matrices(inst: TOPTWInstance,
                                 speed: float = 1.0,
                                 rounding: str = "auto"  # "auto"|"solomon"|"cordeau"|"none"
                                 ) -> Tuple[List[List[float]], List[List[float]]]:
    pts = [(n.x, n.y) for n in inst.nodes]
    n = len(pts)
    if rounding == "auto":
        dec = _auto_rounding_decimals_from_name(inst.path)
    elif rounding == "solomon":
        dec = 1
    elif rounding == "cordeau":
        dec = 2
    else:
        dec = None

    def euclid(a, b): return math.hypot(a[0]-b[0], a[1]-b[1])

    dist = [[0.0]*n for _ in range(n)]
    tmat = [[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                d = 0.0
            else:
                d = euclid(pts[i], pts[j])
                d = _floor_to_decimals(d, dec)
            dist[i][j] = d
            tmat[i][j] = d / speed if speed > 0 else d
    return dist, tmat


## Demand generation

We assign integer `demand` values to non-depot nodes (1..N). The depot (0) keeps demand 0.

We provide three strategies via `mode`:
- `independent`: random in `[lo, hi]` (uniform integers).
- `correlate_profit`: higher profit ⇒ higher demand.
- `depot_distance`: farther from depot ⇒ higher demand.

All strategies return a list `demands` of length `N+1`.



## Choosing a vehicle capacity `Q`

We want capacity to actually matter: total fleet capacity should be *less* than the sum of all customer demand.

Let:
- `K` = number of vehicles.
- `demands` = list of node demands.
- `Q_ratio` = fraction (e.g. 0.25) of total demand that all vehicles combined are allowed to carry.

We set:

\[
Q \approx \left\lfloor \frac{Q\_ratio \cdot \sum_{i>0} q_i}{K} \right\rfloor
\]

and then clamp `Q` so it's at least the largest single-customer demand.



## Exporting a capacitated instance to JSON

We now define a helper that takes:
- the parsed TOPTW instance (without demands),
- a chosen number of vehicles `K`,
- demand generation strategy and parameters,

and writes out an easy-to-read JSON file with:
- all meta fields (`k`, `N`, `Tmax`, ...),
- `Q` (vehicle capacity),
- a full node list including `demand`.

We'll also add a loader to bring that JSON back into the dataclasses.


In [3]:
import json

def load_capacitated_instance(json_path: str) -> TOPTWInstance:
    """Load a JSON file created by convert_to_capacitated_json back into TOPTWInstance."""
    with open(json_path, "r") as f:
        data = json.load(f)

    nodes: List[TOPTWNode] = []
    for nd in data["nodes"]:
        nodes.append(
            TOPTWNode(
                i=nd["i"],
                x=nd["x"],
                y=nd["y"],
                service=nd["service"],
                profit=nd["profit"],
                tw_open=nd["tw_open"],
                tw_close=nd["tw_close"],
                f=nd["f"],
                a=nd["a"],
                aux_list=nd["aux_list"],
                demand=nd.get("demand",0),
            )
        )

    inst = TOPTWInstance(
        path=data.get("name", json_path),
        k=data["k"],
        v=data["v"],
        N=data["N"],
        t=data["t"],
        D=data["D"],
        Q=data["Q"],
        Tmax=data["Tmax"],
        nodes=nodes,
    )
    return inst



## End-to-end demo

Example usage (after you've run all cells above and your original `parse_toptw_instance` is available):

1. Point `SRC_DIR` to your raw benchmark `.txt` files.
2. Generate capacitated JSON instances.
3. Load one back and inspect demands and capacity.

This does **not** overwrite your original data; it writes new `*_cap.json` files.


In [4]:
import os

BASE_DIR = os.getcwd()   # project root (when script is run from repo)
BENCHMARK_FOLDER = os.path.join(BASE_DIR, "benchmarks_cap_json")
demo_file = os.path.join(BENCHMARK_FOLDER, "50_c101_cap.json") # Change to test other files

inst = load_capacitated_instance(demo_file)
print("\nLoaded:", inst.path)
print("Vehicles k:", inst.k)
print("Vehicle capacity Q:", inst.Q)
print("Tmax:", inst.Tmax)
print("#nodes (incl. depot):", len(inst.nodes))
print("Demands:", [n.demand for n in inst.nodes])



Loaded: 50_c101
Vehicles k: 3
Vehicle capacity Q: 21
Tmax: 1236.0
#nodes (incl. depot): 51
Demands: [0, 1, 7, 6, 1, 10, 3, 3, 2, 4, 2, 4, 9, 5, 10, 5, 3, 8, 9, 9, 9, 1, 6, 3, 3, 3, 9, 8, 3, 5, 8, 2, 7, 5, 10, 8, 3, 9, 8, 3, 6, 8, 3, 3, 2, 1, 5, 1, 6, 5, 2]


MIJN IMPLEMENTATIE

In [5]:
from dataclasses import dataclass
from typing import List, Tuple, Optional
import heapq
import numpy as np

@dataclass
class Route:
    """One vehicle route: list of node indices (0 = depot)"""
    nodes: List[int]          # e.g. [0, 5, 12, 0]
    load: int = 0
    time: float = 0.0         # arrival time at the *last* node (before returning)
    profit: float = 0.0
    cost: float = 0.0         # travel distance cost of the route (excluding fixed cost)

    def copy(self) -> "Route":
        return Route(nodes=self.nodes[:], load=self.load,
                     time=self.time, profit=self.profit, cost=self.cost)

2. Feasibility checks op capacity en time windows

In [6]:
def can_insert(inst: TOPTWInstance, dist: List[List[float]], tmat: List[List[float]],
               route: Route, pos: int, cust: int,
               hard_tw: bool = True, beta: float = 0.0) -> Tuple[bool, float, float]:
    """
    Check if customer `cust` can be inserted at position `pos` in `route`.
    Returns (feasible, new_load, new_arrival_at_cust, new_route_time, lateness_at_cust)
    """
    n = len(route.nodes)
    if pos == 0 or pos == n:               # cannot insert before start or after end
        return False, 0, 0, 0, 0

    prev = route.nodes[pos-1]
    nxt  = route.nodes[pos]

    # ----- capacity -----
    new_load = route.load + inst.nodes[cust].demand
    if new_load > inst.Q:
        return False, 0, 0, 0, 0

    # ----- time -----
    arrival_prev = route.time if pos == 1 else \
        route.time + inst.nodes[prev].service + tmat[prev][cust]

    # earliest possible start of service at cust
    earliest = max(arrival_prev, inst.nodes[cust].tw_open)

    if hard_tw and earliest > inst.nodes[cust].tw_close:
        return False, 0, 0, 0, 0

    # service start (wait if we arrive early)
    start_service = earliest
    lateness = 0.0
    if not hard_tw:
        if start_service > inst.nodes[cust].tw_close:
            lateness = start_service - inst.nodes[cust].tw_close
            start_service = start_service   # we still serve at arrival time

    # departure from cust
    departure = start_service + inst.nodes[cust].service

    # time to next node
    arrival_next = departure + tmat[cust][nxt]

    # check TW of the *next* node (must shift its start time)
    nxt_node = inst.nodes[nxt]
    earliest_next = max(arrival_next, nxt_node.tw_open)
    if hard_tw and earliest_next > nxt_node.tw_close:
        return False, 0, 0, 0, 0

    # new route time = arrival at the node *after* the inserted one
    new_route_time = arrival_next + nxt_node.service   # ready to leave nxt

    # optional global route duration limit
    if inst.Tmax is not None:
        return_time = new_route_time + tmat[nxt][0]   # back to depot
        if return_time > inst.Tmax + 1e-6:
            return False, 0, 0, 0, 0

    return True, new_load, start_service, new_route_time, lateness

3. Solution init (greedy)

In [7]:
def greedy_profit_density(inst: TOPTWInstance,
                         dist: List[List[float]],
                         tmat: List[List[float]],
                         K: int,
                         hard_tw: bool = True,
                         f: float = 0.0,
                         beta: float = 0.0) -> List[Route]:
    """
    Classic greedy: at each step insert the customer with highest
    p_i / (d_0i + s_i) into the *cheapest* feasible position of any route.
    """
    unvisited = set(range(1, inst.N+1))
    routes: List[Route] = [Route(nodes=[0, 0]) for _ in range(K)]  # empty routes

    def density(c):
        node = inst.nodes[c]
        d0 = dist[0][c]
        return node.profit / (d0 + node.service + 1e-9)

    while unvisited:
        # 1. candidate with best density
        best_c = max(unvisited, key=density, default=None)
        if best_c is None:
            break

        inserted = False
        best_route_idx = -1
        best_pos = -1
        best_extra_cost = np.inf

        for r_idx, route in enumerate(routes):
            # try every possible slot (between two consecutive nodes)
            for pos in range(1, len(route.nodes)):
                feasible, new_load, _, new_time, lateness = can_insert(
                    inst, dist, tmat, route, pos, best_c, hard_tw, beta)

                if not feasible:
                    continue

                # extra travel cost of the insertion
                prev = route.nodes[pos-1]
                nxt  = route.nodes[pos]
                old_cost = dist[prev][nxt]
                new_cost = dist[prev][best_c] + dist[best_c][nxt]
                extra = new_cost - old_cost

                if extra < best_extra_cost:
                    best_extra_cost = extra
                    best_route_idx = r_idx
                    best_pos = pos

        if best_route_idx == -1:                     # cannot insert anywhere
            unvisited.remove(best_c)
            continue

        # ----- perform insertion -----
        r = routes[best_route_idx]
        cnode = inst.nodes[best_c]
        r.nodes.insert(best_pos, best_c)
        r.load += cnode.demand
        r.profit += cnode.profit
        r.cost += best_extra_cost
        # update arrival time at the node *after* insertion
        _, _, _, new_time, _ = can_insert(inst, dist, tmat, r, best_pos+1, best_c,
                                          hard_tw, beta)
        r.time = new_time

        unvisited.remove(best_c)

    # remove completely empty routes (they cost a fixed fee if f>0)
    
    routes = [r for r in routes if len(r.nodes) > 2]
    return routes

In [8]:
def greedy_regret2(inst: TOPTWInstance,
                   dist: List[List[float]],
                   tmat: List[List[float]],
                   K: int,
                   hard_tw: bool = True,
                   f: float = 0.0,
                   beta: float = 0.0) -> List[Route]:
    """
    Regret-2: at each step insert the customer whose *regret* (difference
    between the best and the second-best insertion cost) is largest.
    The insertion cost is the *penalised* objective:
        Δ = -Δprofit + Δdistance + f*(new vehicle) + β*lateness
    """
    unvisited = set(range(1, inst.N+1))
    routes: List[Route] = [Route(nodes=[0, 0]) for _ in range(K)]

    while unvisited:
        best_c = None
        best_regret = -np.inf
        best_r_idx = best_pos = None
        best_insert_cost = None

        for c in unvisited:
            insert_costs = []                     # list of (Δobj, route_idx, pos)
            for r_idx, route in enumerate(routes):
                for pos in range(1, len(route.nodes)):
                    feasible, new_load, _, new_time, lateness = can_insert(
                        inst, dist, tmat, route, pos, c, hard_tw, beta)
                    if not feasible:
                        continue

                    prev = route.nodes[pos-1]
                    nxt  = route.nodes[pos]
                    old_d = dist[prev][nxt]
                    new_d = dist[prev][c] + dist[c][nxt]
                    delta_dist = new_d - old_d
                    delta_profit = inst.nodes[c].profit
                    delta_f = f if len(route.nodes) == 2 else 0   # new vehicle?
                    delta_obj = -delta_profit + delta_dist + delta_f + beta*lateness

                    insert_costs.append((delta_obj, r_idx, pos))

            if len(insert_costs) == 0:               # cannot be inserted anywhere
                continue

            # sort ascending (smaller Δobj = better insertion)
            insert_costs.sort()
            if len(insert_costs) >= 2:
                regret = insert_costs[1][0] - insert_costs[0][0]
            else:
                regret = 0.0

            if regret > best_regret:
                best_regret = regret
                best_c = c
                best_insert_cost = insert_costs[0]
                best_r_idx, best_pos = insert_costs[0][1], insert_costs[0][2]

        if best_c is None:                          # nothing can be inserted
            break

        # ----- insert the chosen customer -----
        r = routes[best_r_idx]
        cnode = inst.nodes[best_c]
        r.nodes.insert(best_pos, best_c)
        r.load += cnode.demand
        r.profit += cnode.profit
        # recompute route cost & time
        _, _, _, new_time, _ = can_insert(inst, dist, tmat, r, best_pos+1,
                                          best_c, hard_tw, beta)
        r.time = new_time
        r.cost += (best_insert_cost[0] + cnode.profit)   # Δdist part already in delta_obj

        unvisited.remove(best_c)

    # drop empty routes
    routes = [r for r in routes if len(r.nodes) > 2]
    return routes

Objective evaluation

In [9]:
def evaluate_solution(inst: TOPTWInstance,
                      routes: List[Route],
                      dist: List[List[float]],
                      f: float = 0.0,
                      beta: float = 0.0,
                      hard_tw: bool = True,
                      alpha: float = 0.1) -> float:
    """
    Returns the *objective value* (to be maximised):
        Σ profit – α * Σ distance – f * #vehicles – β * Σ lateness
    
    Args:
        alpha: Distance weight. Typical values:
               - 0.01: profit outweighs distance 100:1
               - 0.1:  profit outweighs distance 10:1  (DEFAULT)
               - 0.5:  profit outweighs distance 2:1
    """
    total_profit = sum(r.profit for r in routes)
    total_dist   = sum(r.cost for r in routes)
    n_veh        = len(routes)
    lateness     = 0.0

    if not hard_tw:                     # recompute lateness for soft-TW
        for r in routes:
            cur_time = 0.0
            for i in range(1, len(r.nodes)-1):
                prev = r.nodes[i-1]
                cur  = r.nodes[i]
                cur_time = max(cur_time + inst.nodes[prev].service + dist[prev][cur],
                               inst.nodes[cur].tw_open)
                if cur_time > inst.nodes[cur].tw_close:
                    lateness += cur_time - inst.nodes[cur].tw_close
                cur_time += inst.nodes[cur].service

    return total_profit - alpha * total_dist - f * n_veh - beta * lateness

DEMO

In [10]:
import time
import os

# ------------------------------------------------------------------
# DEMO WITH TIMING AND VERIFICATION
# ------------------------------------------------------------------


dist, tmat = build_distance_time_matrices(inst, rounding="auto")

K = inst.k
hard_tw = True
f = 0.0
beta = 8.0

print(f"Instance: {inst.path}")
print(f"Customers: {inst.N}, Vehicles: {K}, Q = {inst.Q}, Tmax = {inst.Tmax}")
print(f"Total demand: {sum(n.demand for n in inst.nodes[1:])}")
print(f"Total possible profit: {sum(n.profit for n in inst.nodes[1:]):.1f}")
print()

# -------------------------------
# Run greedy + time it
# -------------------------------
start_time = time.time()

# Try regret-2 (usually better than density)
routes = greedy_profit_density(inst, dist, tmat, K, hard_tw, f, beta)

end_time = time.time()

print(f"Greedy finished in {end_time - start_time:.3f} seconds")
print(f"Routes built: {len(routes)} / {K}")
print(f"Customers served: {sum(len(r.nodes)-2 for r in routes)} / {inst.N}")

obj = evaluate_solution(inst, routes, dist, f, beta, hard_tw)
print(f"Objective: {obj:.2f}\n")

# -------------------------------
# Show first 3 routes
# -------------------------------
for i, r in enumerate(routes[:3], 1):
    served = [inst.nodes[j] for j in r.nodes[1:-1]]
    load = sum(c.demand for c in served)
    profit = sum(c.profit for c in served)
    print(f"Route {i}: {' → '.join(str(n.i) for n in [inst.nodes[0]] + served + [inst.nodes[0]])}")
    print(f"   Load: {load}/{inst.Q}, Profit: {profit:.1f}, Nodes: {len(served)}")

Instance: 50_c101
Customers: 50, Vehicles: 3, Q = 21, Tmax = 1236.0
Total demand: 256
Total possible profit: 860.0

Greedy finished in 0.001 seconds
Routes built: 3 / 3
Customers served: 12 / 50
Objective: 266.23

Route 1: 0 → 25 → 16 → 2 → 49 → 44 → 45 → 0
   Load: 21/21, Profit: 140.0, Nodes: 6
Route 2: 0 → 33 → 15 → 21 → 0
   Load: 11/21, Profit: 100.0, Nodes: 3
Route 3: 0 → 46 → 22 → 47 → 0
   Load: 12/21, Profit: 60.0, Nodes: 3


Now use it in a local search

In [11]:
import time
import copy
from typing import List, Tuple, Optional, Dict
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field

# Assumes your existing Route, TOPTWInstance, and helper functions are already defined

@dataclass
class SearchMetrics:
    """Track local search performance"""
    iteration: int = 0
    objective_history: List[float] = field(default_factory=list)
    time_history: List[float] = field(default_factory=list)
    operator_counts: Dict[str, int] = field(default_factory=lambda: {
        'insert': 0, 'relocate': 0, 'swap': 0, '2opt': 0
    })
    improvement_counts: Dict[str, int] = field(default_factory=lambda: {
        'insert': 0, 'relocate': 0, 'swap': 0, '2opt': 0
    })
    customers_served_history: List[int] = field(default_factory=list)
    
    def log_iteration(self, obj: float, elapsed: float, customers: int):
        self.iteration += 1
        self.objective_history.append(obj)
        self.time_history.append(elapsed)
        self.customers_served_history.append(customers)


class LocalSearch:
    def __init__(self, inst: TOPTWInstance, dist: List[List[float]], 
                 tmat: List[List[float]], hard_tw: bool = True, 
                 f: float = 0.0, beta: float = 0.0, alpha: float = 0.1, seed: int = 42):
        self.inst = inst
        self.dist = dist
        self.tmat = tmat
        self.hard_tw = hard_tw
        self.f = f
        self.beta = beta
        self.alpha = alpha  # Distance weight (profit weight is implicitly 1.0)
        self.rng = np.random.default_rng(seed)
        self.metrics = SearchMetrics()
        
    def copy_solution(self, routes: List[Route]) -> List[Route]:
        """Deep copy of solution"""
        return [r.copy() for r in routes]
    
    def get_unvisited(self, routes: List[Route]) -> set:
        """Get set of unvisited customers"""
        visited = set()
        for r in routes:
            visited.update(r.nodes[1:-1])  # exclude depot
        return set(range(1, self.inst.N + 1)) - visited
    
    def recalculate_route_metrics(self, route: Route):
        """Recalculate load, cost, time, and profit for a route"""
        route.load = sum(self.inst.nodes[c].demand for c in route.nodes[1:-1])
        route.profit = sum(self.inst.nodes[c].profit for c in route.nodes[1:-1])
        
        # Calculate cost (distance)
        route.cost = 0.0
        for i in range(len(route.nodes) - 1):
            route.cost += self.dist[route.nodes[i]][route.nodes[i+1]]
        
        # Calculate time (arrival at last customer before depot)
        route.time = 0.0
        for i in range(1, len(route.nodes) - 1):
            prev = route.nodes[i-1]
            curr = route.nodes[i]
            route.time = max(route.time + self.inst.nodes[prev].service + self.tmat[prev][curr],
                           self.inst.nodes[curr].tw_open)
            if i == len(route.nodes) - 2:  # last customer
                route.time += self.inst.nodes[curr].service
    
    def is_route_feasible(self, route: Route) -> bool:
        """Check if route satisfies capacity and time window constraints"""
        # Check capacity
        if route.load > self.inst.Q:
            return False
        
        # Check time windows
        current_time = 0.0
        for i in range(1, len(route.nodes)):
            prev = route.nodes[i-1]
            curr = route.nodes[i]
            
            # Travel time + service at previous
            current_time += self.inst.nodes[prev].service + self.tmat[prev][curr]
            
            # Wait if arriving early
            current_time = max(current_time, self.inst.nodes[curr].tw_open)
            
            # Check if too late
            if current_time > self.inst.nodes[curr].tw_close + 1e-6:
                return False
        
        # Check route duration (time to return to depot)
        if self.inst.Tmax is not None:
            if current_time > self.inst.Tmax + 1e-6:
                return False
        
        return True
    
    # ==================== NEIGHBORHOOD OPERATORS ====================
    
    def try_insert(self, routes: List[Route], current_obj: float) -> Tuple[bool, List[Route], str, float]:
        """Insert operator: find BEST insertion of unvisited customer
        Returns: (feasible, best_routes, message, delta_obj)
        """
        unvisited = self.get_unvisited(routes)
        if not unvisited:
            return False, routes, "", 0.0
        
        best_routes = None
        best_delta = 0.0
        best_message = ""
        
        for customer in unvisited:
            for r_idx in range(len(routes)):
                route = routes[r_idx]
                # Try all positions in route
                for pos in range(1, len(route.nodes)):
                    new_route = route.copy()
                    new_route.nodes.insert(pos, customer)
                    self.recalculate_route_metrics(new_route)
                    
                    if self.is_route_feasible(new_route):
                        new_routes = self.copy_solution(routes)
                        new_routes[r_idx] = new_route
                        new_obj = evaluate_solution(self.inst, new_routes, self.dist, 
                                                   self.f, self.beta, self.hard_tw, self.alpha)
                        
                        delta = new_obj - current_obj
                        if delta > best_delta + 1e-6:
                            best_delta = delta
                            best_routes = new_routes
                            best_message = f"INSERT: Added customer {customer} to route {r_idx} at position {pos} (Δ={delta:.2f})"
        
        if best_routes is not None:
            return True, best_routes, best_message, best_delta
        return False, routes, "", 0.0
    
    def try_relocate(self, routes: List[Route], current_obj: float) -> Tuple[bool, List[Route], str, float]:
        """Relocate operator: find BEST relocation of a customer
        Returns: (feasible, best_routes, message, delta_obj)
        """
        best_routes = None
        best_delta = 0.0
        best_message = ""
        
        for r1_idx in range(len(routes)):
            route1 = routes[r1_idx]
            if len(route1.nodes) <= 2:  # Empty route
                continue
            
            for cust_pos in range(1, len(route1.nodes) - 1):
                customer = route1.nodes[cust_pos]
                
                # Try inserting into all routes (including same route)
                for r2_idx in range(len(routes)):
                    route2 = routes[r2_idx]
                    
                    for insert_pos in range(1, len(route2.nodes)):
                        # Skip if same position in same route
                        if r1_idx == r2_idx and insert_pos == cust_pos:
                            continue
                        
                        # Create new routes
                        new_route1 = route1.copy()
                        new_route1.nodes.pop(cust_pos)
                        self.recalculate_route_metrics(new_route1)
                        
                        new_route2 = route2.copy()
                        # Adjust insert position if same route and removing before insertion
                        adj_insert_pos = insert_pos
                        if r1_idx == r2_idx and cust_pos < insert_pos:
                            adj_insert_pos -= 1
                        new_route2.nodes.insert(adj_insert_pos, customer)
                        self.recalculate_route_metrics(new_route2)
                        
                        if self.is_route_feasible(new_route1) and self.is_route_feasible(new_route2):
                            new_routes = self.copy_solution(routes)
                            new_routes[r1_idx] = new_route1
                            if r1_idx != r2_idx:
                                new_routes[r2_idx] = new_route2
                            else:
                                new_routes[r1_idx] = new_route2
                            
                            new_obj = evaluate_solution(self.inst, new_routes, self.dist, 
                                                       self.f, self.beta, self.hard_tw, self.alpha)
                            
                            delta = new_obj - current_obj
                            if delta > best_delta + 1e-6:
                                best_delta = delta
                                best_routes = new_routes
                                best_message = f"RELOCATE: Moved customer {customer} from route {r1_idx} to route {r2_idx} (Δ={delta:.2f})"
        
        if best_routes is not None:
            return True, best_routes, best_message, best_delta
        return False, routes, "", 0.0
    
    def try_swap(self, routes: List[Route], current_obj: float) -> Tuple[bool, List[Route], str, float]:
        """Swap operator: find BEST swap of two customers between different routes
        Returns: (feasible, best_routes, message, delta_obj)
        """
        if len(routes) < 2:
            return False, routes, "", 0.0
        
        best_routes = None
        best_delta = 0.0
        best_message = ""
        
        for r1_idx in range(len(routes)):
            for r2_idx in range(r1_idx + 1, len(routes)):
                route1, route2 = routes[r1_idx], routes[r2_idx]
                
                if len(route1.nodes) <= 2 or len(route2.nodes) <= 2:
                    continue
                
                for pos1 in range(1, len(route1.nodes) - 1):
                    for pos2 in range(1, len(route2.nodes) - 1):
                        cust1 = route1.nodes[pos1]
                        cust2 = route2.nodes[pos2]
                        
                        # Create new routes with swap
                        new_route1 = route1.copy()
                        new_route1.nodes[pos1] = cust2
                        self.recalculate_route_metrics(new_route1)
                        
                        new_route2 = route2.copy()
                        new_route2.nodes[pos2] = cust1
                        self.recalculate_route_metrics(new_route2)
                        
                        if self.is_route_feasible(new_route1) and self.is_route_feasible(new_route2):
                            new_routes = self.copy_solution(routes)
                            new_routes[r1_idx] = new_route1
                            new_routes[r2_idx] = new_route2
                            new_obj = evaluate_solution(self.inst, new_routes, self.dist, 
                                                       self.f, self.beta, self.hard_tw, self.alpha)
                            
                            delta = new_obj - current_obj
                            if delta > best_delta + 1e-6:
                                best_delta = delta
                                best_routes = new_routes
                                best_message = f"SWAP: Exchanged customer {cust1} (route {r1_idx}) with {cust2} (route {r2_idx}) (Δ={delta:.2f})"
        
        if best_routes is not None:
            return True, best_routes, best_message, best_delta
        return False, routes, "", 0.0
    
    def try_2opt(self, routes: List[Route], current_obj: float) -> Tuple[bool, List[Route], str, float]:
        """2-opt operator: find BEST segment reversal within a single route
        Returns: (feasible, best_routes, message, delta_obj)
        """
        best_routes = None
        best_delta = 0.0
        best_message = ""
        
        for r_idx in range(len(routes)):
            route = routes[r_idx]
            n = len(route.nodes)
            
            if n <= 3:  # Need at least 2 customers to reverse
                continue
            
            # Try all possible segment reversals
            for i in range(1, n - 1):
                for j in range(i + 1, n - 1):
                    # Reverse segment between i and j (inclusive)
                    new_route = route.copy()
                    new_route.nodes[i:j+1] = reversed(new_route.nodes[i:j+1])
                    self.recalculate_route_metrics(new_route)
                    
                    if self.is_route_feasible(new_route):
                        new_routes = self.copy_solution(routes)
                        new_routes[r_idx] = new_route
                        new_obj = evaluate_solution(self.inst, new_routes, self.dist, 
                                                   self.f, self.beta, self.hard_tw, self.alpha)
                        
                        delta = new_obj - current_obj
                        if delta > best_delta + 1e-6:
                            best_delta = delta
                            best_routes = new_routes
                            best_message = f"2OPT: Reversed segment [{i},{j}] in route {r_idx} (Δ={delta:.2f})"
        
        if best_routes is not None:
            return True, best_routes, best_message, best_delta
        return False, routes, "", 0.0
    
    # ==================== MAIN SEARCH ====================
    
    def search(self, initial_routes: List[Route], max_time: float = 120.0, 
               strategy: str = 'best', 
               operator_set: Tuple[str, ...] = ('insert', 'relocate', 'swap', '2opt'),
               shuffle_operators: bool = True,
               verbose: bool = True) -> Tuple[List[Route], SearchMetrics]:
        """
        Main local search.
        
        Args:
            initial_routes: The starting solution
            max_time: Time limit in seconds
            strategy: 'best' (Best-Improvement) or 'first' (First-Improvement)
            operator_set: Tuple of operator names (strings) to use.
            shuffle_operators: Whether to shuffle the operator list each iteration.
            verbose: Print logs
        """
        if strategy not in ['best', 'first']:
            raise ValueError("Strategy must be 'best' or 'first'")
            
        routes = self.copy_solution(initial_routes)
        start_time = time.time()
        
        # Log initial solution
        initial_obj = evaluate_solution(self.inst, routes, self.dist, self.f, self.beta, self.hard_tw, self.alpha)
        initial_customers = sum(len(r.nodes) - 2 for r in routes)
        self.metrics.log_iteration(initial_obj, 0.0, initial_customers)
        
        if verbose:
            print(f"{'='*70}")
            print(f"LOCAL SEARCH STARTED (STRATEGY: {strategy.upper()})")
            print(f"{'='*70}")
            print(f"Operators: {operator_set}")
            print(f"Shuffle: {shuffle_operators}")
            print(f"Initial objective: {initial_obj:.2f}")
            print(f"Max time: {max_time}s")
            print(f"{'='*70}\n")
        
        # Define all *possible* operators
        all_operators = {
            'insert': self.try_insert,
            'relocate': self.try_relocate,
            'swap': self.try_swap,
            '2opt': self.try_2opt
        }
        
        # --- Filter operators based on the hyperparameter ---
        # This is the list of (name, function) tuples we will actually use
        operators_to_use = []
        for op_name in operator_set:
            if op_name in all_operators:
                operators_to_use.append((op_name, all_operators[op_name]))
                # Initialize counts for operators we are actually using
                if op_name not in self.metrics.operator_counts:
                    self.metrics.operator_counts[op_name] = 0
                if op_name not in self.metrics.improvement_counts:
                    self.metrics.improvement_counts[op_name] = 0
            else:
                print(f"Warning: Unknown operator '{op_name}' ignored.")
        
        improvement_found = True
        iteration = 0
        
        while improvement_found:
            improvement_found = False
            elapsed = time.time() - start_time
            
            # Check time limit
            if elapsed >= max_time:
                if verbose:
                    print(f"\n{'='*70}\nTIME LIMIT REACHED ({max_time}s)\n{'='*70}")
                break
            
            current_obj = evaluate_solution(self.inst, routes, self.dist, 
                                            self.f, self.beta, self.hard_tw, self.alpha)
            
            # --- STRATEGY-DEPENDENT LOGIC ---
            
            best_new_routes = None
            best_delta = 0.0
            best_operator = None
            best_message = ""

            # --- Use shuffle hyperparameter ---
            if shuffle_operators:
                self.rng.shuffle(operators_to_use) 
            
            for op_name, op_func in operators_to_use: # Use the filtered list
                self.metrics.operator_counts[op_name] += 1
                
                if verbose and iteration % 5 == 0:
                    print(f"   Evaluating {op_name.upper()} neighborhood...", end='\r')
                
                improved, new_routes, message, delta = op_func(routes, current_obj)
                
                if improved:
                    if strategy == 'best':
                        # BI: Find the best move across ALL operators
                        if delta > best_delta + 1e-6:
                            best_delta = delta
                            best_new_routes = new_routes
                            best_operator = op_name
                            best_message = message
                    
                    elif strategy == 'first':
                        # FI: Take the first improving move we find
                        # (which is the *best* from this *one* neighborhood)
                        best_delta = delta
                        best_new_routes = new_routes
                        best_operator = op_name
                        best_message = message
                        improvement_found = True  # Mark improvement
                        break # <-- Key difference: break operator loop
                
                if time.time() - start_time >= max_time:
                    break # Break operator loop if time's up
            
            # --- END STRATEGY LOGIC ---

            # Apply best move
            # For 'best' strategy, this applies the best move found (if any)
            # For 'first' strategy, this applies the first improving move found
            if best_new_routes is not None:
                routes = best_new_routes
                self.metrics.improvement_counts[best_operator] += 1
                improvement_found = True
                iteration += 1
                
                # Log metrics
                new_obj = evaluate_solution(self.inst, routes, self.dist, 
                                            self.f, self.beta, self.hard_tw, self.alpha)
                current_customers = sum(len(r.nodes) - 2 for r in routes)
                elapsed = time.time() - start_time
                self.metrics.log_iteration(new_obj, elapsed, current_customers)
                
                if verbose:
                    print(f"Iter {iteration:4d} | Time: {elapsed:6.2f}s | Obj: {new_obj:8.2f} | "
                          f"Customers: {current_customers:3d} | {best_message}")

        # Final statistics
        final_obj = evaluate_solution(self.inst, routes, self.dist, self.f, self.beta, self.hard_tw, self.alpha)
        final_customers = sum(len(r.nodes) - 2 for r in routes)
        total_time = time.time() - start_time
        
        if verbose:
            print(f"\n{'='*70}\nLOCAL SEARCH COMPLETED\n{'='*70}")
            print(f"Total iterations: {iteration}")
            print(f"Total time: {total_time:.2f}s")
            print(f"Final objective: {final_obj:.2f}")
            if initial_obj != 0:
                print(f"Improvement: {final_obj - initial_obj:.2f} ({((final_obj/initial_obj - 1)*100):.1f}%)")
            else:
                print(f"Improvement: {final_obj - initial_obj:.2f}")
            print(f"Final customers served: {final_customers}/{self.inst.N}")
            print(f"\nOperator statistics:")
            
            for op_name in operator_set:
                imp_count = self.metrics.improvement_counts.get(op_name, 0)
                att_count = self.metrics.operator_counts.get(op_name, 0)
                print(f"  {op_name.upper():<8}: {imp_count:4d} improvements / {att_count:5d} evaluations")
            print(f"{'='*70}\n")
            
        return routes, self.metrics


# ==================== VISUALIZATION ====================

def plot_search_progress(metrics: SearchMetrics, instance_name: str = ""):
    """Create comprehensive visualization of search progress"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Local Search Progress: {instance_name}', fontsize=14, fontweight='bold')
    
    # Plot 1: Objective value over time
    ax1 = axes[0, 0]
    ax1.plot(metrics.time_history, metrics.objective_history, 'b-', linewidth=2)
    ax1.set_xlabel('Time (seconds)', fontsize=10)
    ax1.set_ylabel('Objective Value', fontsize=10)
    ax1.set_title('Objective Value Over Time', fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Add improvement annotation
    if len(metrics.objective_history) > 1:
        improvement = metrics.objective_history[-1] - metrics.objective_history[0]
        pct_improvement = (improvement / metrics.objective_history[0]) * 100 if metrics.objective_history[0] != 0 else 0
        ax1.text(0.02, 0.98, f'Improvement: {improvement:.1f} ({pct_improvement:.1f}%)', 
                transform=ax1.transAxes, fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Plot 2: Objective value over iterations
    ax2 = axes[0, 1]
    ax2.plot(range(len(metrics.objective_history)), metrics.objective_history, 'g-', linewidth=2)
    ax2.set_xlabel('Iteration', fontsize=10)
    ax2.set_ylabel('Objective Value', fontsize=10)
    ax2.set_title('Objective Value Over Iterations', fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Customers served over time
    ax3 = axes[1, 0]
    ax3.plot(metrics.time_history, metrics.customers_served_history, 'r-', linewidth=2)
    ax3.set_xlabel('Time (seconds)', fontsize=10)
    ax3.set_ylabel('Customers Served', fontsize=10)
    ax3.set_title('Customers Served Over Time', fontsize=11)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Operator effectiveness
    ax4 = axes[1, 1]
    operators = list(metrics.improvement_counts.keys())
    improvements = [metrics.improvement_counts[op] for op in operators]
    attempts = [metrics.operator_counts[op] for op in operators]
    
    x = np.arange(len(operators))
    width = 0.35
    
    bars1 = ax4.bar(x - width/2, attempts, width, label='Attempts', alpha=0.7)
    bars2 = ax4.bar(x + width/2, improvements, width, label='Improvements', alpha=0.7)
    
    ax4.set_xlabel('Operator', fontsize=10)
    ax4.set_ylabel('Count', fontsize=10)
    ax4.set_title('Operator Statistics', fontsize=11)
    ax4.set_xticks(x)
    ax4.set_xticklabels([op.upper() for op in operators], fontsize=9)
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Add success rate text
    for i, (op, att, imp) in enumerate(zip(operators, attempts, improvements)):
        success_rate = (imp / att * 100) if att > 0 else 0
        ax4.text(i, max(att, imp) * 1.05, f'{success_rate:.1f}%', 
                ha='center', fontsize=8)
    
    plt.tight_layout()
    return fig

In [12]:
# ==================== DEMO USAGE ====================

# REPLACE your existing run_local_search_experiment function

# In my_local_search.py
# REPLACE your run_local_search_experiment

def run_local_search_experiment(json_path: str, K_modifier: int = 0, seed: int = 42, 
                              max_time: float = 120.0, alpha: float = 0.1, 
                              strategy: str = 'best',
                              operator_set: Tuple[str, ...] = ('insert', 'relocate', 'swap', '2opt'),
                              shuffle_operators: bool = True,
                              verbose: bool = False, plot: bool = False):
    """
    Complete experiment: load, construct, run LS, and RETURN results.
    """
    
    # Load instance
    inst = load_capacitated_instance(json_path)
    dist, tmat = build_distance_time_matrices(inst, rounding="auto")
    
    # Apply K_modifier
    K = inst.k + K_modifier
    if K <= 0:
        K = 1 # Need at least one vehicle
    
    # Parameters
    hard_tw = True
    f = 0.0
    beta = 0.0
    instance_name = os.path.basename(json_path)
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"Running: {instance_name} | Strategy: {strategy} | Alpha: {alpha} | Seed: {seed}")
        print(f"Operators: {operator_set} | K: {K}")
        print(f"{'='*70}")
    
    # Set seed
    np.random.seed(seed)
    
    # Construct initial solution
    start_construct = time.time()
    initial_routes = greedy_profit_density(inst, dist, tmat, K, hard_tw, f, beta)
    construct_time = time.time()
    
    initial_obj = evaluate_solution(inst, initial_routes, dist, f, beta, hard_tw, alpha)
    
    # Run local search
    ls = LocalSearch(inst, dist, tmat, hard_tw, f, beta, alpha, seed)
    
    # Pass the new parameters to the search method
    final_routes, metrics = ls.search(
        initial_routes, 
        max_time=max_time, 
        strategy=strategy, 
        operator_set=operator_set,
        shuffle_operators=shuffle_operators,
        verbose=verbose
    )
    
    # ... (rest of the function: visualization, result collection) ...
    # (The rest of your function from the previous step is fine)

    # --- Collect Final Results ---
    final_obj = metrics.objective_history[-1] if metrics.objective_history else initial_obj
    final_profit = sum(r.profit for r in final_routes)
    final_distance = sum(r.cost for r in final_routes)
    final_customers = metrics.customers_served_history[-1] if metrics.customers_served_history else 0
    total_time = metrics.time_history[-1] if metrics.time_history else 0
    total_iterations = metrics.iteration - 1 

    results = {
        # 'instance': instance_name, # These are added back in the main runner
        # 'strategy': strategy,
        # 'alpha': alpha,
        # 'seed': seed,
        'initial_obj': initial_obj,
        'final_obj': final_obj,
        'total_time': total_time,
        'iterations': total_iterations,
        'customers_served': final_customers,
        'final_profit': final_profit,
        'final_distance': final_distance,
        'construction_time': construct_time,
    }
    
    return results

In [13]:
import os
import glob
import time
import pandas as pd
from tqdm import tqdm
import itertools  # To help create all combinations

# 1. Path
BASE_DIR = os.getcwd()   # project root (when script is run from repo)
BENCHMARK_FOLDER = os.path.join(BASE_DIR, "benchmarks_cap_json")

# 2. Strategies
STRATEGIES_TO_TEST = ['best', 'first']

# 3. Objective Function Params
ALPHAS_TO_TEST = [0.1, 0.5]  # Maybe just use one alpha for this run

# 4. Statistical Params
SEEDS_TO_TEST = [42, 123, 987]  # <-- NEW: Run 3 times

# 5. Neighborhood Params
OPERATOR_SETS_TO_TEST = [
    # 1. Maximum Quality
    ('insert', 'relocate', 'swap', '2opt'), 
    
    # 2. Balance (High Value)
    ('insert', 'relocate', '2opt'),         
    
    # 3. Speed Core
    ('insert', 'relocate'),
    
    # 4. Intra-Route Focus
    ('2opt',),                               
    
    # 5. Inter-Route Focus
    ('insert', 'relocate', 'swap'),
    
    # 6. Simple Exchange (often fast)
    ('swap',)
]
SHUFFLE_OPERATORS_TO_TEST = [True] # <-- NEW: Test with/without shuffle

# 6. Problem Params
K_MODIFIERS_TO_TEST = [1, 3] # <-- NEW: 0 = instance default, -1 = K-1

# 7. Other settings
MAX_TIME_PER_RUN = 60.0  # Shorter time for more experiments
OUTPUT_CSV = "experiment_results_detailed.csv"

# ==================================================================

def run_local_searches():
    print("Starting TOPTW Local Search Experiment...")
    
    instance_files = glob.glob(os.path.join(BENCHMARK_FOLDER, "*.json"))
    if not instance_files:
        print(f"Error: No .json files found in '{BENCHMARK_FOLDER}'")
        return

    print(f"Found {len(instance_files)} instances.")
    
    # --- Create all experiment combinations ---
    # Use itertools.product to get the Cartesian product of all lists
    experiment_configs = list(itertools.product(
        STRATEGIES_TO_TEST,
        ALPHAS_TO_TEST,
        SEEDS_TO_TEST,
        OPERATOR_SETS_TO_TEST,
        SHUFFLE_OPERATORS_TO_TEST,
        K_MODIFIERS_TO_TEST
    ))
    
    experiment_runs = []
    for instance_path in instance_files:
        for config in experiment_configs:
            strategy, alpha, seed, op_set, shuffle, k_mod = config
            experiment_runs.append({
                'path': instance_path,
                'strategy': strategy,
                'alpha': alpha,
                'seed': seed,
                'op_set': op_set,
                'shuffle': shuffle,
                'k_mod': k_mod
            })

    print(f"Total experiments to run: {len(experiment_runs)}")
    
    all_results = []
    start_total_time = time.time()
    
    for run in tqdm(experiment_runs, desc="Running Experiments"):
        try:
            # Pass all the new parameters to the worker function
            result_dict = run_local_search_experiment(
                json_path=run['path'],
                K_modifier=run['k_mod'],
                seed=run['seed'],
                max_time=MAX_TIME_PER_RUN,
                alpha=run['alpha'],
                strategy=run['strategy'],
                operator_set=run['op_set'],
                shuffle_operators=run['shuffle'],
                verbose=False,
                plot=False
            )
            
            # Add config back to results for the CSV
            result_dict.update(run)
            all_results.append(result_dict)
            
        except Exception as e:
            print(f"!!! ERROR running {os.path.basename(run['path'])} with {run['strategy']} !!!")
            print(f"Error: {e}\n")

    end_total_time = time.time()
    print(f"\n\n{'='*70}")
    print(f"All experiments completed in {(end_total_time - start_total_time) / 60:.2f} minutes.")
    
    if not all_results:
        print("No results to analyze.")
        return

    df = pd.DataFrame(all_results)
    
    # Save results
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"Results saved to '{OUTPUT_CSV}'")
    
    # --- Updated Analysis ---
    
    print("\n--- Strategy Comparison (Averaged over all seeds, alphas, ops) ---")
    # Note: 'final_obj' is now a more reliable average
    strategy_summary = df.groupby('strategy')[['final_obj', 'total_time', 'iterations']].mean()
    print(strategy_summary)
    
    print("\n--- Operator Set Comparison (Averaged over all) ---")
    # We must convert the tuple to a string for groupby
    df['op_set_str'] = df['op_set'].astype(str)
    op_set_summary = df.groupby('op_set_str')[['final_obj', 'total_time', 'iterations']].mean()
    print(op_set_summary)
    
    print(f"\n{'='*70}")
    print("Analysis complete. Open 'experiment_results_detailed.csv' for full details.")

run_local_searches()


Starting TOPTW Local Search Experiment...
Found 58 instances.
Total experiments to run: 8352


Running Experiments:   9%|▉         | 781/8352 [08:45<1:24:57,  1.49it/s]


KeyboardInterrupt: 